In [15]:
import re
import json
import numpy
from typing import List
import nltk
import yfinance as yf
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
import praw
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
def get_env_data():
    with open('.env') as f:
        data = json.load(f)
    return data['client-id'], data['client-secret'], data['username'], data[
        'password']

In [ ]:
def _get_instance() -> praw.Reddit:
    client_id, client_secret, username, password = get_env_data()
    return praw.Reddit(user_agent='adictya',
                       client_id=client_id,
                       client_secret=client_secret,
                       username=username,
                       password=password)

In [27]:
def _comments_on_submission(submission: praw.models.Submission,
                            replace_lim: int = 0) -> List[str]:
    print(
        f' *\tParsing comments with replace limit={replace_lim} on post: {submission.title}'
    )
    submission.comments.replace_more(0)
    return submission.comments.list()

In [79]:
socialNetwork = nx.Graph()
reddit = _get_instance()
subreddit = reddit.subreddit('WallStreetBets')

sid = SentimentIntensityAnalyzer()

positiveWords = [
    'puts', 'green', 'updoot', 'PINS', 'squeeze', 'moon', 'holy', 'mod',
    'bearish', 'up', '🚀'
]
negativeWords = [
    'short', 'red', 'downvote', 'hell', 'curse', 'bulish', 'down', 'drop',
    'crash'
]

hot = reddit.submission(id="n9eiyu")

usernameDict = {}
trustDict = {}

id = -1

In [134]:
for comment in _comments_on_submission(hot):
    
    if (comment.author):
        percentage = 0.5
        trustScore = 0.5
        userName = comment.author.name
        
        if (userName not in usernameDict.keys()):
            id += 1
            usernameDict[userName] = id
            parent_id = id 
            trustDict[id] = trustScore
            socialNetwork.add_node(id, username=userName, trust = trustScore)
            
        else:
            parent_id = usernameDict[userName]
            trustDict[parent_id] += trustScore
            trustDict[parent_id] = min(50,trustDict[parent_id])
            socialNetwork.nodes[parent_id]['trust'] = trustDict[parent_id]
        
        comment.replies.replace_more(0)
        replyList = comment.replies.list()
        
        for reply in replyList:        
            sentiment = 0.5

            if (reply.author and reply.author.name not in usernameDict.keys()):
                userName = reply.author.name
                id += 1
                usernameDict[userName] = id      
                trustDict[id] = trustScore
                socialNetwork.add_node(id,username=userName,trust=trustScore)
                socialNetwork.add_edge(parent_id,id,weight = sentiment * trustScore)
            
            elif(reply.author):
                userName = reply.author.name
                child_id = usernameDict[userName]
                trustDict[child_id] += trustScore
                trustDict[child_id] = min(50,trustDict[child_id])
                socialNetwork.nodes[child_id]['trust'] = trustDict[child_id]
                socialNetwork.add_edge(parent_id,child_id,weight = sentiment * trustScore)


 *	Parsing comments with replace limit=0 on post: What Are Your Moves Tomorrow, May 11, 2021


In [178]:
def returnColor(value):
    if(value>0):
        return "green"
    else:
        return "salmon"
for u,v,a in socialNetwork.edges(data=True):
    if(u+v%5==0):
        socialNetwork.nodes[i]['weight'] *= -1
    socialNetwork.edges[u,v]['color'] = returnColor(a['weight'])

In [180]:
for i,a in socialNetwork.nodes(data=True):
    if(i%5==0):
        socialNetwork.nodes[i]['trust'] *= -1
    if(socialNetwork.nodes[i]['trust'] < 0):
        socialNetwork.nodes[i]['color'] = "salmon"
        socialNetwork.nodes[i]['trust'] *= -1
    else:
        socialNetwork.nodes[i]['color'] = "skyblue"
    print(i,a)

0 {'username': 'silas_p_silas', 'trust': 7.5, 'color': 'salmon'}
1 {'username': 'OG-Outlander', 'trust': 7.0, 'color': 'skyblue'}
2 {'username': 'cmac2200', 'trust': 7.5, 'name': 'cmac2200', 'color': 'skyblue'}
3 {'username': 'wooshock', 'trust': 7.5, 'name': 'wooshock', 'color': 'skyblue'}
4 {'username': 'goooodie', 'trust': 7.5, 'name': 'goooodie', 'color': 'skyblue'}
5 {'username': 'cow_grass', 'trust': 7.0, 'color': 'salmon'}
6 {'username': 'La_Mantequilla', 'trust': 7.0, 'color': 'skyblue'}
7 {'username': 'HookersForDahl2017', 'trust': 7.5, 'name': 'HookersForDahl2017', 'color': 'skyblue'}
8 {'username': 'Cathie_Wood_is_Bae', 'trust': 9.0, 'name': 'Cathie_Wood_is_Bae', 'color': 'skyblue'}
9 {'username': 'alongcameapoem', 'trust': 50, 'name': 'alongcameapoem', 'color': 'skyblue'}
10 {'username': 'bsr92', 'trust': 7.0, 'color': 'salmon'}
11 {'username': 'ButtfuckerTim', 'trust': 7.5, 'name': 'ButtfuckerTim', 'color': 'skyblue'}
12 {'username': 'redskady', 'trust': 7.0, 'color': 'sky

In [181]:
from bokeh.io import output_notebook, show, save
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

output_notebook()

title = 'StocksReddit Social Information Network'
HOVER_TOOLTIPS = [("name", "@username"),("trust","@trust")]
edge_highlight_color = 'black'

plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

network_graph = from_networkx(socialNetwork, nx.spring_layout, scale=10, center=(0, 0))

network_graph.node_renderer.glyph = Circle(size="trust", fill_color="color",line_width=0.5,
                                           line_alpha=0.5)

network_graph.edge_renderer.glyph = MultiLine(line_color='color'
                                              ,line_alpha=1, line_width=1)


plot.renderers.append(network_graph)

show(plot)

Loading BokehJS ...

8.0

In [ ]:
socialNetwork = nx.Graph()
for comment in _comments_on_submission(hot):
    stocks = re.findall("[A-Z]{1,5}", comment.body)
    stockData = ""
    for stock in stocks:
        stockData = yf.Ticker(stock).info
        if len(stockData) > 1:
            break
    if (len(stockData) > 1):
        try:
            prevClose = stockData["previousClose"]
            currOpen = stockData["open"]
        except Exception as e:
            prevClose = 1
            currOpen = 0
        percentage = ((currOpen - prevClose) / prevClose) * 100
#         trustScore = sid.polarity_scores(
#             comment.body)["compound"] * percentage
        trustScore = 0.5
        if (comment.author.name not in myDict.keys()):
            id = id + 1
            myDict[comment.author.name] = id
            parent_id = id
            socialNetwork.add_node(id,trust = trustScore)
        else:
            parent_id = myDict[comment.author.name]
        comment.replies.replace_more(0)
        replyList = comment.replies.list()
        for reply in replyList:
#             sentiment = sid.polarity_scores(reply.body)["compound"]
            sentiment = 0.5
            if (reply.author and reply.author.name not in myDict.keys()):
                id = id + 1
                myDict[reply.author.name] = id
                socialNetwork.add_node(id)
#                 oldScore = socialNetwork[parent_id][id]['weight']
                socialNetwork.add_edge(parent_id,id,weight = sentiment * trustScore)
            else:
                childId = myDict[reply.author.name]
                socialNetwork[parent_id][childId] = sentiment * trustScore
#                 oldScore = socialNetwork[parent_id][childId]['weight']
                socialNetwork.add_edge(parent_id,childId,weight = sentiment * trustScore)

In [ ]:
options = {
    'with_labels': True,
#     'node_color': 'black',
    'node_size': 140,
    'width': 2,
}
plt.figure(3,figsize=(12,12))
nx.draw(socialNetwork,**options)
plt.show()

In [ ]:
moiDick = {}
